In [14]:
import zipfile

In [10]:
#First we perform a curl request for the curl data
!curl -o prismdata.zip https://ftp.prism.oregonstate.edu/normals_4km/ppt/PRISM_ppt_30yr_normal_4kmM4_all_bil.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
﻿
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0 21.1M    0 32768    0     0  72029      0  0:05:07 --:--:--  0:05:07 72495
 59 21.1M   59 12.6M    0     0  9151k      0  0:00:02  0:00:01  0:00:01 9168k
100 21.1M  100 21.1M    0     0  11.7M      0  0:00:01  0:00:01 --:--:-- 11.7M


In [17]:
#Next we must unzip the files and save them to a file in our work environment
zip_file_path = 'prismdata.zip'
extracted_folder = r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5571 Lab2_2\PRISM"

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder)

print(f'All files have been extracted to {extracted_folder}')

All files have been extracted to C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5571 Lab2_2\PRISM


In [16]:
#Now we create a Mosaic dataset
arcpy.management.CreateMosaicDataset(
    in_workspace=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5571 Lab2_2\GIS 5571 Lab2_2.gdb",
    in_mosaicdataset_name="Mosaic",
    coordinate_system='GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
    num_bands=None,
    pixel_type="",
    product_definition="NONE",
    product_band_definitions=None
)

<Result 'C:\\Users\\conno\\OneDrive\\Documents\\ArcGIS\\Projects\\GIS 5571 Lab2_2\\GIS 5571 Lab2_2.gdb\\Mosaic'>

In [18]:
#Then, we add the PRISM data to the mosaic
arcpy.management.AddRastersToMosaicDataset(
    in_mosaic_dataset="Mosaic",
    raster_type="Raster Dataset",
    input_path=r"'C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5571 Lab2_2\PRISM'",
    update_cellsize_ranges="UPDATE_CELL_SIZES",
    update_boundary="UPDATE_BOUNDARY",
    update_overviews="NO_OVERVIEWS",
    maximum_pyramid_levels=None,
    maximum_cell_size=0,
    minimum_dimension=1500,
    spatial_reference=None,
    filter="",
    sub_folder="SUBFOLDERS",
    duplicate_items_action="ALLOW_DUPLICATES",
    build_pyramids="NO_PYRAMIDS",
    calculate_statistics="NO_STATISTICS",
    build_thumbnails="NO_THUMBNAILS",
    operation_description="",
    force_spatial_reference="NO_FORCE_SPATIAL_REFERENCE",
    estimate_statistics="NO_STATISTICS",
    aux_inputs=None,
    enable_pixel_cache="NO_PIXEL_CACHE",
    cache_location=r"C:\Users\conno\AppData\Local\ESRI\rasterproxies\Mosaic"
)

<Result 'Mosaic'>

In [19]:
#Next, we calculate a couple fields
arcpy.management.CalculateField(
    in_table=r"Mosaic\Footprint",
    field="Variable",
    expression='"Mosaic"',
    expression_type="PYTHON3",
    code_block="",
    field_type="TEXT",
    enforce_domains="NO_ENFORCE_DOMAINS"
)
arcpy.management.CalculateField(
    in_table=r"Mosaic\Footprint",
    field="Timestamp",
    expression="DateAdd(Date(1991,0,1), $feature.OBJECTID-1, 'year')",
    expression_type="ARCADE",
    code_block="",
    field_type="DATE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

<Result 'Mosaic\\Footprint'>

In [21]:
#Now after that prep work, we can use the adjusted mosaic to create multidimensional info
arcpy.md.BuildMultidimensionalInfo(
    in_mosaic_dataset="Mosaic",
    variable_field="Variable",
    dimension_fields="Timestamp # #",
    variable_desc_units=None,
    delete_multidimensional_info="NO_DELETE_MULTIDIMENSIONAL_INFO"
)

<Result 'Mosaic'>

In [22]:
#That multidimensional data can then be used to construct a multidimensional raster
arcpy.md.MakeMultidimensionalRasterLayer(
    in_multidimensional_raster="Mosaic",
    out_multidimensional_raster_layer="Mosaic_MultidimLayer",
    variables="'Mosaic'",
    dimension_def="ALL",
    dimension_ranges=None,
    dimension_values=None,
    dimension="",
    start_of_first_iteration="",
    end_of_first_iteration="",
    iteration_step=None,
    iteration_unit="",
    template='-125.020833333333 24.0624999997935 -66.4791666661985 49.9375000000005 GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]',
    dimensionless="DIMENSIONS",
    spatial_reference=None
)

<Result 'Mosaic_MultidimLayer'>

In [7]:
#And finally, we use the multidimensional raster to construct a space time cube
arcpy.stpm.CreateSpaceTimeCubeMDRasterLayer(
    in_md_raster="Mosaic_MultidimLayer",
    output_cube=r"C:\Users\conno\OneDrive\Documents\ArcGIS\Projects\GIS 5571 Lab2_2\PRISM_SPT.nc",
    fill_empty_bins="ZEROS"
)

<Result 'C:\\Users\\conno\\OneDrive\\Documents\\ArcGIS\\Projects\\GIS 5571 Lab2_2\\PRISM_SPT.nc'>